# Certificates

Public-private keypairs in notebook [Cryptography I](https://colab.research.google.com/drive/1qHZslGbMXCDSXAnYyHLtB9G-Tbwml_q0#scrollTo=eU7wgqS3l7nO) were not associated with any identity (*Subject*). Association between public keys and subjects is described by *certificates*.

The most widely used certificate format is X509. Unfortunately, there are many different file formats for storing X509 certificates and the corresponding private keys.

The specific file format to use in a given application depends on the software that has to use the certificate (or the private key). The corresponding commands are quite complex and intricate (a [small subset of them](https://www.xolphin.com/support/OpenSSL/Frequently_used_OpenSSL_Commands), just to have an idea). You can find just a few very basic examples below.


Execute this command before executing the examples below.

In [ ]:
!cd ~/; openssl rand -writerand .rnd

## Generate a self-signed certificate

Generate a 1024 bit public-private keypair and a self-signed certificate (valid for 365 days, with RSA public key cryptography, 2048 bits key length).

The private key will be stored in file *privateKey.key* (without any password protection). The association between public key and Subject, i.e., the self-signed certificate, will be stored in file *certificate.crt*.

The Subject for the certificate is a set of fields named Country Name, Organizational Unit, Common Name, etc (full details below, in the "Play yourself" section). You can specify the Subject in any way you want. If you omit the -subj option, then *openssl* will ask this information interactively and you can answer in any way you want.

Thus, **anyone can create a self-signed certificate for anyone else**.


In [ ]:
!openssl req -x509 -nodes -days 365 -newkey rsa:2048 -keyout privateKey.key -out certificate.crt  -subj "/C=IT/CN=Presidente della Repubblica Italiana/OU=Quirinale"

Dump content of the certificate in textual form.

You will see that X509 certificates store a lot more information than (Subject, Public Key of Subject, Issuer, Expiration Date, Signature by Issuer). The usage of this information is beyond the scope of this notebook. For our purposes, the interesting thing is that you can specify the Subject/Issuer of a self-signed certificate in any way you want.

In [ ]:
!!openssl x509 -in certificate.crt -noout -text

Have a look at the private key (stored in the default format of the openssl command used).

In [ ]:
!cat privateKey.key

## Play yourself

Try to generate a self-signed certificate with some "interesting" Subject.

The Subject fields that you can use are:
* CN: CommonName ("your" name or a DNS name)
* OU: OrganizationalUnit
* O: Organization
* L: Locality
* S: StateOrProvinceName
* C: CountryName
* emailAddress: email address

You need not specify all of them (the set of those that are required and of those that are optional depend on the software that will use the certificate, i.e., on the purpose of the certificate).

##Download web server certificates

Fetch certificate from the HTTPS server at *google.com* and store it in file *googlecert.pem* in *pem* format

In [ ]:
! openssl s_client -connect google.com:443 -showcerts </dev/null | openssl x509 -outform pem > googlecert.pem

Dump pem file in text format.


In [ ]:
!openssl x509 -in googlecert.pem -noout -text


If you try to find the Issuer of the above certificate in the KeySet/TrustSet of your operating system, probably you will not find it. In other words, you will not find a self-signed certificate for the Issuer in (the KeySet/Trust/set of) your operating system.

The reason is beyond the scope of this notebook. From our point of view, just pretend that a self-signed certificate for the Issuer of every certificate can always be found in (the KeySet/Trust/set of) your operating system.

This specific topic is treated in a different course. Just as a "curiosity": in most real cases, the Issuer of the certificate is the Subject of *another* certificate that is *not* self-signed. The Issuer of this second certificate is the Subject of a *third* certificate that is *not* self-signed, and so on until a certificate that is self-signed. Only this last certificate must be stored in (the KeySet/Trust/set of) your operating system.

In other words, in most real cases, certificates form a *chain* in which the Issuer of a certificate is the Subject of the previous one, except for the first certificate in the chain that is self-signed (called the *anchor* of the chain).

## Play yourself

Try to generate a self-signed certificate with a Subject identical to the one in the certificate of the HTTPS server of senato.it (or of any other "interesting" HTTPS server).

# Digital signature

Create a file that will be signed. We use a text file because it can be displayed easily. You can use any type of file you want: pdf, jpg, mp3...

In [ ]:
!echo "kakà, shevchenko, gullit, van basten" > players.txt
!cat players.txt

## Signing side (owner of the private key)

Create a private-public keypair and a self-signed certificate for an "interesting" Subject.

In [ ]:
!openssl req -x509 -nodes -days 365 -newkey rsa:2048 -keyout privateKey.key -out certificate.crt  -subj "/C=IT/CN=Presidente della Repubblica Italiana/OU=Quirinale"

Input:
* file to sign (players.txt)
* private key (privateKey.key)

Output:
* signature (players.txt.signature)

Construction of a digital signature with openssl requires two separate commands (what these commands do and what the purpose of the intermediate temporary file is, are irrelevant to us).

Of course, the file that contains the private key has to be protected carefully. Furthermore, in practice, its usage must be protected by a password (this has not been done here for simplicity; see similar examples in notebook [Cryptography I](https://colab.research.google.com/drive/1qHZslGbMXCDSXAnYyHLtB9G-Tbwml_q0#scrollTo=GdfO7X1VKHy3)).

In [ ]:
!openssl dgst -sha256 -sign privateKey.key -out /tmp/sign.sha256 players.txt
!openssl base64 -in /tmp/sign.sha256 -out players.txt.signature

Have a look at the signature: just a sequence of bytes of fixed length. The length depends on the specific signature algorithm (and key length) used.

In [ ]:
!cat players.txt.signature

## Verifying side (anyone with a certificate for the signer)

Input:
* signed file (players.txt)
* signature (players.txt.signature)
* certificate

Output:
* OK or not OK

How the input data have been obtained by the verifying side is irrelevant. This is a crucial point.

Input data may have reached the verifying side at the same time or at different times. They could have travelled through the same channel or through different channels. Such channels need not be trusted, which is also a crucial point.

In most practical applications, the three pieces of input data are merged together in a *single file* whose internal structure is defined by some standard (just as a curiosity: [this](https://www.adobe.com/devnet-docs/etk_deprecated/tools/DigSig/Acrobat_DigitalSignatures_in_PDF.pdf) is a standard for signed PDF files). The verification procedure will extract the three required pieces from the single file used.



Verification of a digital signature with openssl requires three steps.

First you have to extract the public key from the input certificate.

In [ ]:
!openssl x509 -pubkey -noout -in certificate.crt  > pubkey.pem

Then you have to execute two separate commands (what these commands do and what the purpose of the intermediate temporary file is, are irrelevant to us).

In [ ]:
!openssl base64 -d -in players.txt.signature -out /tmp/sign.sha256
!openssl dgst -sha256 -verify pubkey.pem -signature /tmp/sign.sha256 players.txt

## Understanding authentication

The output of the verification is ok if and only if:
* The signed file is identical to the one that was signed, and
* The signature is identical to the one that was constructed when signing, and
* The certificate contains the public key associated with the private key that was used for signing.

In this case, integrity and authentication hold. Authentication means what has been just written: the signature was constructed with the private key associated with the public key used in the verification procedure.

Authentication of a digital signature has *nothing* to do with any form of "identity in the real world" such as, e.g., name and surname, fiscal code, role in an organization, DNS name, email address or whatever.

The example above has used a certificate whose Subject was "*/C=IT/CN=Presidente della Repubblica Italiana/OU=Quirinale*". Verification succeeded but the signature was *not* constructed by the real entity that everyone would associate with that Subject.

Authentication of a digital signature can be bound to a "real identity" only under several additional hypotheses and several non-cryptographic procedures.

In a nutshell: the certificate used by the verifying side must not be self-signed; its Issuer must have verified the "real identity" of the Subject successfully; the public key of the Issuer must be available to the verifying side.

## Real applications

Usage of *openssl* for real applications of certificates and digital signature requires:
* Given a public-private keypair for a Subject "S1" and a self-signed certificate for a Subject "S2", the ability of generating a certificate with Subject="S2" and Issuer="S1" (i.e., the actions that "S1" acting as a certification authority would do).
* Given a certificate not self-signed, the ability of verifying that the certificate was issued by an Issuer in a given KeySet/TrustSet.

Executing these actions with *openssl* is very complex and quite tricky.

We do not show any example.

Furthermore, as pointed out above, real applications usually embeds file, signature and certificate in a single file of some standard format. Constructing such files (on the signing side) and using them (on the verifying side) is also very complex.

This can be done by using *openssl* as a library invoked by specialized software rather than as a command line as done in this notebook.

## Play yourself

* Create a self-signed certificate (and keypair) for some "interesting" Subject (e.g., Alberto Bartoli).
* Create some "interesting" file (e.g., a textual file with an excellent grade in Computer Networks).
* Sign that file.
* Verify the signature.
* Modify something in the file and/or the signature and verify again.

